In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

# Data Preprocessing

In [ ]:
img_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=.2)

training_set = img_datagen.flow_from_directory(
        'PATH_TO_YOUR_DATASET_DIR',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical',
        subset='training'
)

valid_set = img_datagen.flow_from_directory(
        'PATH_TO_YOUR_DATASET_DIR',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical',
        subset='validation'
)

In [ ]:
print(training_set.image_shape, valid_set.image_shape)
print(training_set.batch_size, valid_set.batch_size)
print(training_set.labels.shape, valid_set.labels.shape)

In [ ]:
def img_set_as_arr(dataset):
    imgs = np.concatenate([dataset.next()[0] for i in range(len(dataset))])
    labels = np.concatenate([dataset.next()[1] for i in range(len(dataset))])
    return (imgs, labels)

In [ ]:
train_imgs, train_labels = img_set_as_arr(training_set)
print(train_imgs.shape, train_labels.shape)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(9):
    img = train_imgs[i]
    label = train_labels[i]
    plt.subplot(3,3, i+1)
    plt.imshow(img)

In [ ]:
valid_imgs, valid_labels = img_set_as_arr(test_set)
print(valid_imgs.shape, valid_labels.shape)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(9):
    img = valid_imgs[i]
    label = valid_imgs[i]
    plt.subplot(3,3, i+1)
    plt.imshow(img)

# Building the CNN

In [ ]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=4, activation='softmax'))

# Training the CNN

In [ ]:
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
cnn.fit(training_set, validation_data=valid_set, epochs = 10)

# Testing the CNN Model

In [ ]:
from keras.preprocessing import image

In [ ]:
def predict_img_class(img_path):
    img = image.load_img(img_path, target_size=(64,64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    result = cnn.predict(img / 255.0)[0]
    classes = list(training_set.class_indices)
    prob_dist = {}
    for i in range(len(result)):
        prob_dist[classes[i]] = (result[i] * 100).round(2)
    return (classes, prob_dist)

In [ ]:
classes, prob_dis = predict_img_class('PATH_TO_IMAGE_YOU_WANT_TO_TEST')
print('Probability of Each Shape: \n', prob_dis)

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(plt.imread('PATH_TO_IMAGE_YOU_WANT_TO_TEST'))
plt.title('Testing Image Reveal')
plt.show()